### 2. 进程


#### 2.3 进程通信
大概有 7 种常见的进程间的通信方式。

1. 管道/匿名管道(Pipes) ：用于具有亲缘关系的父子进程间或者兄弟进程之间的通信。只存在于内存中的文件

   * 匿名管道由于没有名字，只能用于亲缘关系的进程间通信。
   * 为了克服这个缺点，提出了有名管道。有名管道严格遵循先进先出 (FIFO)。有名管道以磁盘文件的方式存在，可以实现本机任意两个进程通信。

2. 信号(Signal) ：信号是一种比较复杂的通信方式，用于通知接收进程某个事件已经发生；

3. 消息队列(Message Queuing) ：消息队列是消息的链表, 具有特定的格式, 存放在内存中并由消息队列标识符标识。管道和消息队列的通信数据都是先进先出的原则。与管道（无名管道：只存在于内存中的文件；命名管道：存在于实际的磁盘介质或者文件系统）不同的是消息队列存放在内核中，只有在内核重启(即，操作系统重启)或者显示地删除一个消息队列时，该消息队列才会被真正的删除。消息队列可以实现消息的随机查询, 消息不一定要以先进先出的次序读取, 也可以按消息的类型读取. 比FIFO更有优势。消息队列克服了信号承载信息量少，管道只能承载无格式字 节流以及缓冲区大小受限等缺。

4. 信号量(Semaphores) ：信号量是一个计数器，用于多进程对共享数据的访问，信号量的意图在于进程间同步。这种通信方式主要用于解决与同步相关的问题并避免竞争条件。

5. 共享内存(Shared memory) ：使得多个进程可以访问同一块内存空间，不同进程可以及时看到对方进程中对共享内存中数据的更新。这种方式需要依靠某种同步操作，如互斥锁和信号量等。可以说这是最有用的进程间通信方式。

6. 套接字(Sockets) : 此方法主要用于在客户端和服务器之间通过网络进行通信。套接字是支持TCP/IP的网络通信的基本操作单元，可以看做是不同主机之间的进程进行双向通信的端点，简单的说就是通信的两方的一种约定，用套接字中的相关函数来完成通信过程。


#### 2.5 死锁

产生死锁的原因主要是：
1. 因为系统资源不足。
2. 进程运行推进的顺序不合适。
3. 资源分配不当等。

如果系统资源充足，进程的资源请求都能够得到满足，死锁出现的可能性就很低，否则就会因争夺有限的资源而陷入死锁。其次，进程运行推进顺序与速度不同，也可能产生死锁。

产生死锁的四个必要条件：
1. 互斥条件：一个资源每次只能被一个进程使用。
2. 请求与保持条件：一个进程因请求资源而阻塞时，对已获得的资源保持不放。
3. 不剥夺条件: 进程已获得的资源，在未使用完之前，不能强行剥夺。
4. 循环等待条件: 若干进程之间形成一种头尾相接的循环等待资源关系。

这四个条件是死锁的必要条件，只要系统发生死锁，这些条件必然成立，而只要上述条件之
一不满足，就不会发生死锁。

### 3. 线程

#### 3.1 线程状态

1. 初始(NEW)：新创建了一个线程对象，但还没有调用start()方法。

2. 运行(RUNNABLE)：Java线程中将就绪（ready）和运行中（running）两种状态笼统的称为“运行”。 线程对象创建后，其他线程(比如main线程）调用了该对象的 start() 方法。
   该状态的线程位于可运行线程池中，等待被线程调度选中，获取CPU的使用权，此时处于就绪状态（ready）。就绪状态的线程在获得 CPU时间片后 变为运行中状态（running）。

3. 阻塞(BLOCKED)：**表示线程阻塞于锁。注意和进程的区别，进程是IO阻塞。**

4. 等待(WAITING)：进入该状态的线程需要等待其他线程做出一些特定动作（通知或中断）。

5. 超时等待(TIMED_WAITING)：该状态不同于WAITING，它可以在指定的时间后自行返回。

6. 终止(TERMINATED)：表示该线程已经执行完毕。

#### 3.2 线程通信 Python

从一个线程向另一个线程发送数据最安全的方式可能就是使用 queue 库中的队列了。创建一个被多个线程共享的 Queue 对象，这些线程通过使用 put() 和 get() 操作来向队列中添加或者删除元素。 

In [1]:
"""
消费者在读到这个特殊值之后立即又把它放回到队列中，将之传递下去。这样，所有监听这个队列的消费者线程就可以全部关闭了。 
"""
import time
from queue import Queue
from threading import Thread

class Consumer(Thread):
   def __init__(self, queue):
       # 从python3 开始，继承有了极大的改善
       super().__init__()
       # 我喜欢用单下划线去标识私有变量
       self._queue = queue

   # 继承了Thread类后，需要重写run方法来自定义事件
   def run(self):
       while True:
           # msg即是我们说的消息，也是仓库中的货物
           msg = self._queue.get()
           # 我会在生产者中加入quit关键字，保证程序能够自动退出
           if isinstance(msg, str) and msg == 'quit':
               break
           print(f"I'm a thread, and I received {msg}!!")
       print('Bye byes!')


def producer():
   # 生产者生产一个新的队列，用于存放消息
   queue = Queue()
   # 初始化一个消费者实例
   worker = Consumer(queue)
   # 开启消费者线程
   worker.start()  
   start_time = time.time()
   # 退出条件
   while time.time() - start_time < 5:
       queue.put('something at %s' % time.time())
       time.sleep(1)
   queue.put('quit')
   worker.join()


if __name__ == '__main__':
   producer()

I'm a thread, and I received something at 1605627500.8116422!!
I'm a thread, and I received something at 1605627501.816792!!
I'm a thread, and I received something at 1605627502.8203049!!
I'm a thread, and I received something at 1605627503.822793!!
I'm a thread, and I received something at 1605627504.823749!!
Bye byes!


### 4. 异步非阻塞 I/O

阻塞，非阻塞

1. 具体到系统底层，就是读写事件，而当读写事件没有准备好时，必然不可操作，如果不用非阻塞的方式来调用，那就得阻塞调用了，事件没有准备好，那就只能停止等待，线程被挂起。

2. 非阻塞就是，事件没有准备好，马上返回 EAGAIN，告诉我们，事件还没准备好。

异步，非异步：

如果持续的来询问事件是否准备好，也会是一笔不小的开销。

所以，才会有了异步非阻塞的事件处理机制，具体到系统调用就是像 select/poll/epoll/kqueue 这样的系统调用。它们提供了一种机制，让我们可以同时监控多个事件，调用他们是阻塞的，但可以设置超时时间，在超时时间之内，如果有事件准备好了，就返回。这种机制正好解决了我们上面的两个问题，拿epoll为例(在后面的例子中，我们多以 epoll 为例子，以代表这一类函数)，当事件没准备好时，放到 epoll 里面，事件准备好了，我们就去读写，当读写返回 EAGAIN 时，我们将它再次加入到 epoll 里面。

这样，只要有事件准备好了，我们就去处理它，只有当所有事件都没准备好时，才在 epoll 里面等着。这样，我们就可以并发处理大量的并发请求了。

### 5. 锁

1. CAS, Compare-And-Swap: 比较并交换值。基本思路为，检测 ptr 指向的值是否和 expected 相等，如果是，则更新 ptr 的值为新值，获取了锁。否则，就什么也不做。因此，锁被持有的时候，竞争锁的线程会自旋。

#### 5.1 Python 如何加锁

### 6. 内存虚拟化？分段和分页

Reference：[内存管理](https://github.com/CyC2018/CS-Notes/blob/master/notes/%E8%AE%A1%E7%AE%97%E6%9C%BA%E6%93%8D%E4%BD%9C%E7%B3%BB%E7%BB%9F%20-%20%E5%86%85%E5%AD%98%E7%AE%A1%E7%90%86.md)

#### 虚拟内存

   虚拟内存的目的是为了让物理内存扩充成更大的逻辑内存，从而让程序获得更多的可用内存。

   为了更好的管理内存，操作系统将内存抽象成地址空间。每个程序拥有自己的地址空间，这个地址空间被分割成多个块，每一块称为一页。这些页被映射到物理内存，但不需要映射到连续的物理内存，也不需要所有页都必须在物理内存中。当程序引用到不在物理内存中的页时，由硬件执行必要的映射，将缺失的部分装入物理内存并重新执行失败的指令。（缺页）
   

#### 分页

   内存管理单元（MMU）管理着地址空间和物理内存的转换，其中的页表（Page table）存储着页（程序地址空间）和页框（物理内存空间）的映射表。

   一个虚拟地址分成两个部分，一部分存储页面号，一部分存储偏移量。每个进程有着自己的地址空间。
   

#### 页面置换算法

1. LFU
2. LRU
3. FIFO


#### 分段

分段的做法是把每个表分成段，一个段构成一个独立的地址空间。每个段的长度可以不同，并且可以动态增长。



### 7. ToDo

* 什么是原子操作？什么情况原子操作会失败呢？

"原子操作(atomic operation)是不需要 synchronized"，这是多线程编程的老生常谈了。所谓原子操作是指不会被线程调度机制打断的操作；这种操作一旦开始，就一直运行到结束，中间不会有任何上下文的切换。

* 进程与线程的区别（linux的跟python以及java的不一样？）内存管理

* 共享数据是堆内存还说栈内存？堆和栈存的东西有什么区别？

* 什么时候需要关心线程安全问题？举个例子？

* 如何避免线程安全问题？Python 对线程如何加锁？

* Python 哪些队列是线程安全的？能用于进程通信吗？

* 内存的缓存机制？LRU 是什么意思？如何实现？

* Linux的 watch和ctrl+c是怎么实现的

* 终端里 curl 后过程 HTTP完整过程 

* Compare and Swap，一种乐观锁的实现，可以称为"无锁"(lock-free)，CAS 由于要保证原子性无法由 JVM 本身实现，需要调用对应 OS 的指令(这块其实我不了解细节)

* 指针和引用的区别

* 多线程怎么实现同步

* 介绍一下虚拟内存

* 动态绑定的底层原理

* 进程调度算法

* Epoll和其他两个IO复用的区别
   - 多线程的IO复用和单线程的IO复用有什么区别，为什么要用多线程呢
   - Redis为什么高效，为什么它不用多线程呢
   - 水平触发和边缘触发的区别和使用场景

* 讲一下操作系统的内存管理
   - 地址转换
   - CPU的缓存，为什么要设置L1 L2 L3缓存（面试官想考察程序的局部性原理）

* 手撕：实现阻塞队列（生产者消费者模型），且要求取的时候先取优先级最大的
   我先直接用优先队列，后面面试官让我再实现最大堆

* 进程和线程的区别
   - 多线程怎么通信
   - 条件变量
   - 多线程之间怎么保证安全，用过哪种
   - 两个线程能在cpu中同时运行吗
   

### 8. 虚拟化

    虚拟化是为了让操作系统更易用。

#### 8.1 虚拟化 CPU

    虚拟化 CPU 的目的是为了更好的利用 CPU，例如，产生多道程序“同时”运行在单个 CPU 核的假象。

##### 进程

    进程就是运行中的程序，是操作系统为正在运行的程序提供的抽象，是资源分配和调度的单位。
    
    因此，进程的机器状态（组成）：
    
    1. 内存。进程要执行的指令存在内存中，读取和写入的数据也在内存中，因此进程可访问的内存是该进程的一部分。
    
    2. 寄存器。许多指令会明确的读取和更新寄存器，如：程序计数器，告诉进程执行哪个指令；栈指针和相关的帧指针，管理函数参数栈、局部变量和返回地址。
    
    
**进程如何被操作系统创建**
    
    ——————————————————
    |                |
    |       代码     |
    |     静态数据    |
    |       堆       |
    |                |
    |                |
    |                |
    |                |
    |_______栈_______|
        
        进程内存示意图


    1. 将代码和所有静态数据加载到内存中，加载到进程的地址空间

    
    2. 程序使用栈存放局部变量，函数参数和返回地址。操作系统分配这些内存，提供给进程。
    
    3. 操作系统也为程序的堆分配一些内存，主要是用于显式请求的动态分配数据。
    

**进程状态**

1. 创建状态(new) ：进程正在被创建，尚未到就绪状态。

2. 就绪状态(ready) ：进程已处于准备运行状态，即进程获得了除了处理器之外的一切所需资源，一旦得到处理器资源(处理器分配的时间片)即可运行。

3. 运行状态(running) ：进程正在处理器上上运行(单核CPU下任意时刻只有一个进程处于运行状态)。

4. 阻塞状态(waiting) ：又称为等待状态，进程正在等待某一事件而暂停运行如等待某资源为可用或等待 IO 操作完成。即使处理器空闲，该进程也不能运行。

5. 结束状态(terminated) ：进程正在从系统中消失。可能是进程正常结束或其他原因中断退出运行。


**数据结构 PCB **

    PCB，是操作系统存储进程的描述信息所用的数据结构。
    
    PCB 的内容主要有：
    
    1. 标示符。描述本进程的唯一标示符，用来区别其他进程
    2. 状态。任务状态，退出代码，退出信号等。
    3. 优先级。相对于其他进程的优先级。
    4. 程序计数器。程序中即将被执行的下一条指令的地址。
    5. 内存指针。包括程序代码和进程相关数据的指针，还有和其他进程共享的内存块的指针
    6. 上下文数据。进程执行时处理器的寄存器中的数据
    7. I／O状态信息。包括显示的I/O请求,分配给进程的I／O设备和被进程使用的文件列表。
    8. 记账信息。可能包括处理器时间总和，使用的时钟数总和，时间限制，记账号等

**父子进程***

1. fork

调用 fork() 创建的子进程与父进程几乎一样，但是父进程中，fork()返回的是子进程的 PID，子进程返回的是 0，由此可以区分以编程。子进程有自己的地址空间。

2. wait

父进程 调用 wait() 等待子进程结束后，再继续执行

3. exec

fork() 适用于让子进程运行相同程序的拷贝，如果想要运行不同的程序，则可以使用  exec()，先通过 fork() 创建子进程，在子进程代码区 使用 exec() 加载其它的可执行文件。


**进程调度算法**

1. FIFO：先进先出

缺点：一些耗时较少的进程被排在重量级进程之后，得不到及时的执行。（护航效应）

2. SJF：最短任务优先

缺点：只有进程同时到达才能评估并达到理想效果，但是现实不是这样，对于后来的进程，仍有可能出现（护航效应）。

3. STCF：最短完成时间优先

向 SJF 增加抢占，每当新的进程进入，比较当前进程的剩余时间和新进程的剩余时间。

4. Round-Robin：轮转

在一个时间片内运行一个进程，然后切换到运行队列中的下一个进程。

缺点：假如时间片太短，上下文切换的开销带来的影响会增大。

5. MLFQ：多级反馈队列

    基本规则：存在多个运行队列，每个队列有不同的优先级。优先执行高优先级的进程，同一队列进行轮转。
    
    那么，如何设置优先级？
    
    答：进程的优先级是动态的，例如，一个工作不断放弃 CPU 去等待用户的I/O，则可能是交互型进程，会让它保持较高的优先级。假如一个进程长时间占有 CPU，则 MLFQ 有可能会降低其优先级。
    
    规则：
    
    
    1. 进程 A > 进程 B 的优先级，则运行 A
    2. 进程 A = 进程 B 的优先级，则轮转运行 A 和 B
    3. 新的进程进入系统，置为最高优先级
    4. 进程使用完一个时间片，降低优先级，在此之前主动放弃 CPU 则保持优先级不变
    5. 经过一段时间 S，当前所有工作都会重新进入最高优先级队列。避免饥饿的问题。如果 S 太高，长的工作会饥饿，太低则 I/O 进程得不到合适的 CPU 时间比例。
    6. 记录一个进程在某一层消耗的总时间，避免被进程愚弄（总在时间片结束前主动放弃）。对进程每一层的执行时间施行配额，用完了配额就一定降低优先级
    
    用例：
    
    1. 如果来了一个进程：如果是短的进程，则会在被移入最低优先级之前运行完毕，否则会被慢慢降入低优先级。
    2. I/O 密集型进程：这样的进程经常会在时间片结束前主动放弃CPU，不降低它的优先级而是保持
    

**多处理器调度下的问题**

    多处理器架构：多处理器共享内存的问题主要是缓存的问题。
    

    **缓存** ： 缓存是基于**局部性**的概念，分为时间局部性和空间局部性。时间局部性是指，当一个数据被访问后，很有可能不久后再次访问；空间局部性是指，访问地址为 x 的数据后，有可能接着访问 x 周围的数据。
    
    1. 缓存一致性
    
    多 CPU 下的缓存，可能会出现**缓存一致性**问题，例如 CPU1 上的程序从内存地址 A 中读取数据，由于不再 CPU1 的缓存中，所以系统直接访问内存，得到值 D1，程序随后在缓存中将其更新为值 D2，还没来得及写入内存，这时系统中断了程序执行，将其交给 CPU2，这时只会读取到旧值。
    
    如何解决：

    答：缓存一致性协议。
    
    硬件层面的缓存一致性协议（Cache Coherence Protocol），最出名的就是Intel 的MESI协议，MESI协议保证了每个缓存中使用的共享变量的副本是一致的。

    MESI的核心的思想是：当CPU写数据时，如果发现操作的变量是共享变量，即在其他CPU中也存在该变量的副本，会发出信号通知其他CPU将该变量的缓存行置为无效状态，因此当其他CPU需要读取这个变量时，发现自己缓存中缓存该变量的缓存行是无效的，那么它就会从内存重新读取。

    在MESI协议中，每个缓存可能有有4个状态，它们分别是：

    M(Modified)：这行数据有效，数据被修改了，和内存中的数据不一致，数据只存在于本Cache中。

    E(Exclusive)：这行数据有效，数据和内存中的数据一致，数据只存在于本Cache中。

    S(Shared)：这行数据有效，数据和内存中的数据一致，数据存在于很多Cache中。

    I(Invalid)：这行数据无效。
    
    
    2. 缓存亲和度
    
    一个进程在某个 CPU 上运行，会在该 CPU 的缓存中维护许多状态，下次在相同的 CPU 上运行时才会运行的更快。
    
    3. 多队列调度
    
    为每个 CPU 单独维护调度队列。工作量较少的队列不定期“偷看”其他队列是否比自己工作量少多，是的话就“偷窃”一个或多个工作，实现负载均衡，这个技术叫做工作窃取。


#### 8.2 虚拟化内存

    虚拟化内存，是为了程序更好更容易的使用内存，主要目标为：
    
    1. 透明。使运行的程序不能察觉到内存被虚拟化。哪怕是 C 指针的地址也是虚拟的。
    
    2. 效率。
    
    3. 保护。确保进程收到保护，不受其它进程的影响。

    操作系统需要提供一个易用的物理内存抽象，这个抽象叫做“地址空间”，是运行的程序所看到的系统中的内存。
    
    一个进程的地址空间，包含程序指令（代码），位于地址空间的顶部。其次是堆（顶部）管理动态分配的、用户管理的内存，栈（底部）保存当前函数的调用信息，分配空间给局部变量，传递参数和函数返回值。
    
    
**内存使用**

    1. double* ptr = (double*) malloc(10 * sizeof(double)) 传入要申请的堆空间的大小，返回指向新空间的指针。
    
    2. free(ptr) 传入要释放的内存的指针。
    

**地址转换**

    既然进程使用的是虚拟地址，则就涉及基于硬件的从虚拟地址到物理地址的地址转换。操作系统需要在转换时，设置好硬件，以便正确转换，因此操作系统需要记录被占有的和空闲的内存位置。
    
    这个硬件单元一般叫做 MMU。MMU(Memory Management Unit)主要用来管理虚拟存储器、物理存储器的控制线路，同时也负责虚拟地址映射为物理地址，以及提供硬件机制的内存访问授权、多任务多进程操作系统。
    
    最简单的即为在MMU中引入两个寄存器：基址寄存器和界限寄存器。这样：物理地址 = 虚拟地址 + 基础地址。界限寄存器提供保护。
    
**分段**

     不是只在 MMU 中引入一对基址寄存器和界限寄存器，还是给地址空间内每个逻辑段一对。一个段只是地址空间里的一个连续定长的区域，典型的地址空间3个逻辑不同的段：
     
     1. 代码段
     2. 栈
     3. 堆
     
     分段的机制使得操作系统可以将不同的段放到不同的物理内存区域，从而避免虚拟地址空间内未使用部分占用物理内存。所谓的 SegmentFault 段错误就是在分段的机器上发生了非法的内存访问。如今，在非分段的机器上，试图访问非法地址也会导致段异常或段错误。
     
     我们可以基于如此的虚拟地址进行转换：
     
     ——————————————————
    ｜ 段号 ｜ 段内偏移量｜
     ——————————————————
     
     存在的问题：
     
     1. 在所有地址都保持一个增长方向时才可以，然而栈是反向增长的，因此它可能是从 32KB 增长回到 24KB。因此，**硬件还需记录段的增长方向**。
     
     2. 为了支持内存共享，那么每个段一个增加保护位，来标识这个程序是否能够读写该段，或执行其中的代码，抑或是只能进行读取。这样，同样的代码可以被多个进程共享，同时保证不被隔离。
     
     3. 分段带来的问题，进程上下午切换时，也应该进行段寄存器段的保存与恢复。
     
     
**分页**

分段技术将空间切成不同长度的分片以后，容易导致外部碎片，随着时间退役，分配内存会比较困难。

分页，不是将地址空间分割成几个不同长度的逻辑段，而是分割成固定大小的单元，每个单元称之为一页。相应地，我们把物理内存看成是定长槽块的阵列，叫做页帧。每个这样的页帧包含一个虚拟内存页。

    **一个例子：**
    
    假如，一个页大小 16 字节，操作系统需要将 64 字节的小地址空间放到 8 页的物理地址空间，只需要找到 4 个空闲页，将虚拟页映射到这4个物理页帧即可。为了为地址空间每个虚拟页保存地址转换，记录在物理内存中的实际位置，操作系统通常会为每个进程保存一个数据结构，成为**页表**。
    
    页表是一个每进程的数据结构！
    
    我们可以基于如此的虚拟地址进行转换：
     
     ——————————————————————
    ｜ 虚拟页号 ｜ 页内偏移量 ｜
     ——————————————————————
     
     在转换时，在页表中查询虚拟页号对应的实际的物理页面，就可以得到真实的物理地址。由于页表可以变得很大，因此不是存在 MMU 中，而是将每个进程的页表存在内存中。页表中的项称为页表项，有效位用于指示特定地址转换是否有效；保护位用于指示页是否可以读取、写入或执行；存在位表示该页在内存中还说物理磁盘中；脏位表示页面被加载进内存后是否被修改过。
     
     由于页表的存在，当页表很大时，地址转换会很慢。 -- **TLB**，快速地址转换。
     
     为了加快地址转换，增加了硬件 TLB。它就是对频繁发生的虚拟到物理地址转换的硬件缓存。对每次内存访问，硬件先检查 TLB，再检查页表。
     
     **TLB基本流程：**
     
     1. 从虚拟地址提取虚拟页号 VPN。
     2. 检查 TLB 中是否有该 VPN的转换映射。如果用，命中，快速完成转换
     3. 如果没有命中，查找页表寻找转换映射，更新 TLB，系统重新尝试该指令，得到处理。
     
     TLB 中的存储项结构：
     
     ————————————————————————————————————
    ｜ 虚拟页号VPN ｜ 物理页帧 PFN ｜ 其它位 ｜
     ————————————————————————————————————
     
     TLB是全关联的，硬件会并行的进行查找，因此速度很快。
     
     *TLB 中的有效位指示该映射是否有效，PTE 的有效位指示该页有没有被进程申请使用。*
     
     
     **TLB替换策略：**
     
     1. LRU：最近最少使用
     
     
**多级页表**
    
    如何去掉页表中的所有无效区域，而不是将它们全部保留在内存中？使用多级页表将线性页表变成类似树结构。
    
    多级页表的基本思想为：

1. 将页表分为页大小的单元

2. 如果整页的页表项无效，则完全不分配该页的页表

3. 使用页目录来追踪页表的页是否有效，页目录会记录页表的页的位置，以及页表的整个页是否包含有效页。

    多级页表是有成本的，TLB 未命中时需要加载两次，一次用于页目录，一次用于PTE本身。
    

**空闲内存管理**

    free() 的调用只需要传入指针，而不需要传入指针对应的空间大小，这就是因为操作系统对内存进行了管理。在堆上追踪空闲和已分配空间的数据结构通常被称为**空闲列表**（不一定是真的列表形式）。
    
    如果要管理的空闲空间大小是不一致的，那将主要面临外部碎片问题。例如，总的空间 32KB 被已分配空间16KB 分割成两个 8 KB空间，那么一个分配16 KB 的请求就会失败。这样的情况常发生在用户级别的内存分配库或者操作系统使用分段的形式实现虚拟内存。
    
    基本机制：
    
1. 分割与合并。需要分配内存时，分割满足要求的空闲内存区域；释放内存时，操作系统有时会其去其它空闲区域合并以满足较大的内存分配请求。

2. 追踪以分配空间的大小。大多数分配程序会在一个已分配的区域加上 Header，写入例如 Size 等管理空间所需的信息。

3. 当堆空间耗尽，再向操作系统申请更大的空间，通常是经过系统调用，例如 UNIX 系统中的 sbrk，让堆增长。一般是会找到空闲的物理页，将它映射到进程的地址空间去，并返回新的堆末尾地址。
    
    
    **匹配策略**：
    
1. 最优匹配。每次遍历空闲列表，找到最合适的。

优点：避免空间浪费；缺点：性能代价高


2. 最差匹配。与最优匹配相反，表现糟糕。


3. 首次匹配。返回第一个满足要求的块。

优点：速度快；缺点：使得空闲列表开头外部碎片多

4. 下次匹配。不同于首次匹配从列表头开始查找，而是维护一个指针，记录上次匹配查找结束的位置。

优点：和首次匹配性能相近，避免遍历查找。缺点：需要额外空间。


    **伙伴系统**

    将空闲空间看作2的幂次方大小的空间。每次分配，空闲空间都被一分为二，直到刚好满足要求。如果一块内存被释放，会检查伙伴释放被释放，是的话就合并并向上检查。


**上下文切换**

    当操作系统决定切换一个进程，会为当前进程保存一些寄存器的值（例如，存到内核栈），并为即将执行的进程恢复一些寄存器的值，这个过程叫做上下文切换。由于 TLB 中的内容只对当前进程有效，因此在进行进程切换时：
    1. 要么将 TLB 的内容清空（将有效位置为 0
    2. 一些系统增加了标识符，表示此映射被哪个进程使用，就可以共享 TLB



**交换空间**

    为了超越物理内存，利用较慢的设备，透明的提供巨大虚拟地址空间的假象，需要进行空间交换：在硬盘上开辟一部分空间用于物理页的移入和移出。这样的空间叫做交换空间，因为我们将内存中的页交换到其中，并在需要的时候交换回去。为了达到这个目的，操作系统需要记住给定页的硬盘物理地址。
    
    交换空间的大小，决定了系统在某一时刻能够使用的最大内存页数。因此，PTE 内需要存在位来表示页是否在物理内存中。访问不在物理内存中的页就会引发页错误 PageFault。
    
    操作系统会使用 PTE 内的某些位来存储硬盘地址，当操作系统接收到页错误时，会在 PTE 中查找地址，将I/O请求发送到硬盘，将页读取到内存中。所以，发生页错误的时候，进程将进入阻塞状态。流程：
    
1. 页错误
2. 寻找可用的物理页帧
3. 如找不到，则需要抛弃当前内存中的某些页帧
4. I/O 请求读取页
5. 更新页表，操作系统重试指令，TLB 未命中
6. 更新 TLB，操作系统重试命中

    那么，何时发生页交换？大多数操作系统会设置高水位线 High Watermark,HW 和低水位线 Low Watermark,LW。当操作系统发现少于 LW 个页可用时，后台负责释放内存的线程会开始运行，直到有 HW 个可用页。这个后台线程有时被称为交换守护线程。
    

**页替换策略**

1. 最优策略，替换将来被用的最少的页。

2. FIFO

3. 随机

4. LRU


**脏页，抖动**

* 脏页就是读入内存并已经被修改的页，替换时置换脏页就需要将其写入硬盘，代价很大。因此，修改位标志其是否被修改。

* 假如系统将时间主要花在页的置换，这种情况称之为抖动。


### 9. 并发

对于多进程程序，多线程程序所可能出现的问题的处理。